In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# syft absolute
import syft as sy
from syft.service.code.user_code import UserCode
from syft.service.job.job_stash import Job
from syft.service.request.request import Request

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
request = [
    r for r in high_client.requests if r.code.service_func_name == "large_sample_bed17f"
][0]

In [ ]:
request

In [ ]:
def execute_request(client_high, request) -> dict:
    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found"

    func_name = request.code.service_func_name
    api_func = getattr(client_high.code, func_name, None)
    if api_func is None:
        return "Code name was not found on the client."

    job = api_func(blocking=False)
    return job

In [ ]:
job = execute_request(high_client, request)

In [ ]:
job

In [ ]:
assert isinstance(job, Job)

In [ ]:
result = job.wait()
assert len(result) == 10000

In [ ]:
job_info = job.info(result=True)
response = request.deposit_result(job_info, approve=True)
assert isinstance(response, Job)

In [ ]:
request = [
    r for r in high_client.requests if r.code.service_func_name.startswith("popular")
][0]

In [ ]:
job = execute_request(high_client, request)
assert isinstance(job, Job)

In [ ]:
result = job.wait()
assert len(result) == 10000

In [ ]:
job_info = job.info(result=True)
response = request.deposit_result(job_info, approve=True)
assert isinstance(response, Job)

In [ ]:
server.land()